In [1]:
from datasets import load_dataset

dataset = load_dataset("lksy/ru_instruct_gpt4")
dataset = dataset["train"]
dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: ru_instruct_gpt4/default
Found cached dataset ru_instruct_gpt4 (/home/kosenko/.cache/huggingface/datasets/lksy___ru_instruct_gpt4/default/0.0.1/b2cfa50a05a4e665ead398fce73de411b1fd8791d282289a2c79eede7097b1b4)
100%|██████████| 1/1 [00:00<00:00, 482.99it/s]


Dataset({
    features: ['instruction', 'input', 'output', 'full_output'],
    num_rows: 15056
})

In [2]:
def filter(string):
    wrong_substrings = [
        "стих",
        'Проанализируй фразу и определи, какой философ ей бы отдавал предпочтение. "Существование предшествует сущности.',
        
    ]

    for item in wrong_substrings:
        if item in string:
            return False

    return True


def fix_string(string):
    wrong_substrings = [
        "Input:",
        '"На заре в деревне было тихо и спокойно."',
        '"магазине яблоки в купил"',
        'Выход:',
        "Вход:"
    ]

    for item in wrong_substrings:
        string = string.replace(item, "")

    return string

entire_string = ""
new_dataset = []
for i in range(len(dataset)):
    instruction = dataset[i]["instruction"]
    inp = dataset[i]["input"]
    full_output = dataset[i]["full_output"]
    output = dataset[i]["output"]

    instruction = f"{instruction} {inp}"
    instruction = fix_string(instruction)

    result = None
    if not full_output is None:
        result = full_output
    else:
        result = output
    result = fix_string(result)
    
    total_string = instruction + result
    if filter(total_string):
        entire_string += f"{instruction}\n"
        entire_string += f"{result}\n"
        new_dataset.append({
            "prompt": instruction,
            "solution": result
        })
        entire_string += f"--------------------------------------------\n"
        entire_string += f"--------------------------------------------\n"
        # print(instruction)
        # print(result)

    # print("-" * 100)
    # print("-" * 100)
# print(entire_string)

In [3]:
from datasets import Dataset
dataset = Dataset.from_list(new_dataset)

In [4]:
dataset.push_to_hub("dim/ru_instruct_gpt4")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:03<00:00,  3.43s/it]
